In [2]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.datasets import KarateClub

df=pd.read_json('53e9aa95b7602d970340bc5c_emb.json', lines='true',orient='records')
df2=pd.read_json('53e9aa95b7602d970340bc5c.json', lines='true',orient='records')

In [3]:
df3=df[df['id'].isin(df2['id'])]
del df
df=df3

In [6]:
edgelist=[]
for i in df.iterrows():
    if i[1]['references']!=None:
        for j in i[1]['references']:
                edgelist.append((i[1]['id'],j))
el=pd.DataFrame(edgelist,columns=['source','target'])
el = el[el['target'].isin(df['id'])]
G = nx.from_pandas_edgelist(el, 'source', 'target',create_using=nx.DiGraph())
for i in df['id']:
    G.add_node(i)

In [9]:
papers=df['id'].tolist()
label_encoder = LabelEncoder()

labled_papers = label_encoder.fit_transform(papers)
df['label']=labled_papers

print("Numeric Labels:", len(labled_papers))
print("Label Mapping:", len(list(label_encoder.classes_)))

print(df.shape)
print(G.number_of_nodes())  

Numeric Labels: 2291
Label Mapping: 2291
(2291, 9)
2291


In [10]:
el['sL']=label_encoder.transform(el['source'])
el['tL']=label_encoder.transform(el['target'])

In [11]:
edge_list=[]
edge_list.append(el['sL'].tolist())
edge_list.append(el['tL'].tolist())
edge_list=torch.tensor(edge_list)

In [12]:
dataset = KarateClub()
type(dataset)

torch_geometric.datasets.karate.KarateClub

In [ ]:
data = dataset[0]
data.x=torch.tensor(df['emb'].tolist())
data.edge_index = torch.tensor(edge_list)

In [15]:
from typing import Callable, List, NamedTuple, Optional, Tuple

import torch
import torch.nn.functional as F
from torch import Tensor

from torch_geometric.utils import coalesce, scatter, softmax


class UnpoolInfo(NamedTuple):
    edge_index: Tensor
    cluster: Tensor
    batch: Tensor
    new_edge_score: Tensor


class EdgePooling(torch.nn.Module):
    r"""The edge pooling operator from the `"Towards Graph Pooling by Edge
    Contraction" <https://graphreason.github.io/papers/17.pdf>`__ and
    `"Edge Contraction Pooling for Graph Neural Networks"
    <https://arxiv.org/abs/1905.10990>`__ papers.

    In short, a score is computed for each edge.
    Edges are contracted iteratively according to that score unless one of
    their nodes has already been part of a contracted edge.

    To duplicate the configuration from the `"Towards Graph Pooling by Edge
    Contraction" <https://graphreason.github.io/papers/17.pdf>`__ paper, use
    either :func:`EdgePooling.compute_edge_score_softmax`
    or :func:`EdgePooling.compute_edge_score_tanh`, and set
    :obj:`add_to_edge_score` to :obj:`0.0`.

    To duplicate the configuration from the `"Edge Contraction Pooling for
    Graph Neural Networks" <https://arxiv.org/abs/1905.10990>`__ paper,
    set :obj:`dropout` to :obj:`0.2`.

    Args:
        in_channels (int): Size of each input sample.
        edge_score_method (callable, optional): The function to apply
            to compute the edge score from raw edge scores. By default,
            this is the softmax over all incoming edges for each node.
            This function takes in a :obj:`raw_edge_score` tensor of shape
            :obj:`[num_nodes]`, an :obj:`edge_index` tensor and the number of
            nodes :obj:`num_nodes`, and produces a new tensor of the same size
            as :obj:`raw_edge_score` describing normalized edge scores.
            Included functions are
            :func:`EdgePooling.compute_edge_score_softmax`,
            :func:`EdgePooling.compute_edge_score_tanh`, and
            :func:`EdgePooling.compute_edge_score_sigmoid`.
            (default: :func:`EdgePooling.compute_edge_score_softmax`)
        dropout (float, optional): The probability with
            which to drop edge scores during training. (default: :obj:`0.0`)
        add_to_edge_score (float, optional): A value to be added to each
            computed edge score. Adding this greatly helps with unpooling
            stability. (default: :obj:`0.5`)
    """
    def __init__(
        self,
        in_channels: int,
        edge_score_method: Optional[Callable] = None,
        dropout: Optional[float] = 0.0,
        add_to_edge_score: float = 0.5,
    ):
        super().__init__()
        self.in_channels = in_channels
        if edge_score_method is None:
            edge_score_method = self.compute_edge_score_softmax
        self.compute_edge_score = edge_score_method
        self.add_to_edge_score = add_to_edge_score
        self.dropout = dropout

        self.lin = torch.nn.Linear(2 * in_channels, 1)

        self.reset_parameters()

    def reset_parameters(self):
        r"""Resets all learnable parameters of the module."""
        self.lin.reset_parameters()

    @staticmethod
    def compute_edge_score_softmax(
        raw_edge_score: Tensor,
        edge_index: Tensor,
        num_nodes: int,
    ) -> Tensor:
        r"""Normalizes edge scores via softmax application."""
        return softmax(raw_edge_score, edge_index[1], num_nodes=num_nodes)

    @staticmethod
    def compute_edge_score_tanh(
        raw_edge_score: Tensor,
        edge_index: Optional[Tensor] = None,
        num_nodes: Optional[int] = None,
    ) -> Tensor:
        r"""Normalizes edge scores via hyperbolic tangent application."""
        return torch.tanh(raw_edge_score)

    @staticmethod
    def compute_edge_score_sigmoid(
        raw_edge_score: Tensor,
        edge_index: Optional[Tensor] = None,
        num_nodes: Optional[int] = None,
    ) -> Tensor:
        r"""Normalizes edge scores via sigmoid application."""
        return torch.sigmoid(raw_edge_score)

    def forward(
        self,
        x: Tensor,
        edge_index: Tensor,
        batch: Tensor,
    ) -> Tuple[Tensor, Tensor, Tensor, UnpoolInfo]:
        r"""Forward pass.

        Args:
            x (torch.Tensor): The node features.
            edge_index (torch.Tensor): The edge indices.
            batch (torch.Tensor): The batch vector
                :math:`\mathbf{b} \in {\{ 0, \ldots, B-1\}}^N`, which assigns
                each node to a specific example.

        Return types:
            * **x** *(torch.Tensor)* - The pooled node features.
            * **edge_index** *(torch.Tensor)* - The coarsened edge indices.
            * **batch** *(torch.Tensor)* - The coarsened batch vector.
            * **unpool_info** *(UnpoolInfo)* - Information that is
              consumed by :func:`EdgePooling.unpool` for unpooling.
        """
        e = self.compute_edge_score(x, x,edge_index, x.size(0))
        print(f'Edge Score: {e}')
        print(f'Edge Score: {e.shape}')
        e = F.dropout(e, p=self.dropout, training=self.training)
        e = e + self.add_to_edge_score

        x, edge_index, batch, unpool_info,cluster = self._merge_edges(
            x, edge_index, batch, e)

        return x, edge_index, batch, unpool_info,cluster

    def _merge_edges(
        self,
        x: Tensor,
        edge_index: Tensor,
        batch: Tensor,
        edge_score: Tensor,
    ) -> Tuple[Tensor, Tensor, Tensor, UnpoolInfo]:

        cluster = torch.empty_like(batch)
        perm: List[int] = torch.argsort(edge_score, descending=True).tolist()

        # Iterate through all edges, selecting it if it is not incident to
        # another already chosen edge.
        mask = torch.ones(x.size(0), dtype=torch.bool)

        i = 0
        new_edge_indices: List[int] = []
        edge_index_cpu = edge_index.cpu()
        for edge_idx in perm:
            source = int(edge_index_cpu[0, edge_idx])
            if not bool(mask[source]):
                continue

            target = int(edge_index_cpu[1, edge_idx])
            if not bool(mask[target]):
                continue

            new_edge_indices.append(edge_idx)

            cluster[source] = i
            mask[source] = False

            if source != target:
                cluster[target] = i
                mask[target] = False

            i += 1

        # The remaining nodes are simply kept:
        j = int(mask.sum())
        cluster[mask] = torch.arange(i, i + j, device=x.device)
        i += j

        # We compute the new features as an addition of the old ones.
        # new_x = scatter_mean(x, cluster, dim=0, dim_size=i)
        new_x = scatter(x, cluster, dim=0, dim_size=cluster.max().item()+1, reduce='mean')
        new_edge_score = edge_score[new_edge_indices]
        if int(mask.sum()) > 0:
            remaining_score = x.new_ones(
                (new_x.size(0) - len(new_edge_indices), ))
            new_edge_score = torch.cat([new_edge_score, remaining_score])
        new_x = new_x * new_edge_score.view(-1, 1)

        new_edge_index = coalesce(cluster[edge_index], num_nodes=new_x.size(0))
        new_batch = x.new_empty(new_x.size(0), dtype=torch.long)
        new_batch = new_batch.scatter_(0, cluster, batch)

        unpool_info = UnpoolInfo(edge_index=edge_index, cluster=cluster,
                                 batch=batch, new_edge_score=new_edge_score)

        return new_x, new_edge_index, new_batch, unpool_info ,cluster

    def unpool(
        self,
        x: Tensor,
        unpool_info: UnpoolInfo,
    ) -> Tuple[Tensor, Tensor, Tensor]:
        r"""Unpools a previous edge pooling step.

        For unpooling, :obj:`x` should be of same shape as those produced by
        this layer's :func:`forward` function. Then, it will produce an
        unpooled :obj:`x` in addition to :obj:`edge_index` and :obj:`batch`.

        Args:
            x (torch.Tensor): The node features.
            unpool_info (UnpoolInfo): Information that has been produced by
                :func:`EdgePooling.forward`.

        Return types:
            * **x** *(torch.Tensor)* - The unpooled node features.
            * **edge_index** *(torch.Tensor)* - The new edge indices.
            * **batch** *(torch.Tensor)* - The new batch vector.
        """
        new_x = x / unpool_info.new_edge_score.view(-1, 1)
        new_x = new_x[unpool_info.cluster]
        return new_x, unpool_info.edge_index, unpool_info.batch

    def __repr__(self) -> str:
        return f'{self.__class__.__name__}({self.in_channels})'


In [16]:


def compute_edge_cosine_similarity(
        source_node_features: Tensor,
        target_node_features: Tensor,
        edge_index: Tensor,
        num_nodes: Optional[int] = None,
    ) -> Tensor:
        r"""Computes cosine similarity score for edges."""
        source_features = source_node_features[edge_index[0]]
        target_features = target_node_features[edge_index[1]]
        
        # Compute cosine similarity
        cosine_sim = F.cosine_similarity(source_features, target_features, dim=1)
        
        return cosine_sim

In [17]:
class motif_ig(torch.nn.Module):
    def __init__(self,num_features):
        super(motif_ig, self).__init__()
        self.pool_layer = EdgePooling(num_features, edge_score_method=compute_edge_cosine_similarity,add_to_edge_score=2,dropout=0.5)

    def forward(self, x, edge_index, batch):
        x1 = self.pool_layer(x, edge_index, batch)
        x2 = self.pool_layer(x1[0], x1[1], x1[2])
        x3 = self.pool_layer(x2[0], x2[1], x2[2])
        x4 = self.pool_layer(x3[0], x3[1], x3[2])
        x5 = self.pool_layer(x4[0], x4[1], x4[2])
        x6 = self.pool_layer(x5[0], x5[1], x5[2])
        x7 = self.pool_layer(x6[0], x6[1], x6[2])
        return x1, x2, x3 , x4, x5 , x6, x7


In [53]:
from collections import defaultdict
def cumilating(tensors): 
    clusters = {i: [i] for i in range(len(tensors[0]))}

    def find(cluster, node):
        while cluster[node] != node:
            node = cluster[node]
        return node

    for tensor in tensors:
        new_clusters = defaultdict(list)
        for idx, value in enumerate(tensor):
            new_clusters[int(value)].extend(clusters[idx])
        
        clusters = {key: list(set(value)) for key, value in new_clusters.items()}

    final_clusters = list(clusters.values())

    return final_clusters

In [21]:
l=[0]*data.num_nodes
batch = torch.tensor(l, dtype=torch.long)

motifs=motif_ig(data.num_features)
x=motifs(data.x, data.edge_index, batch)

tensors = [i[3][1] for i in x]
i=cumilating(tensors)

Edge Score: tensor([0.4157, 0.3419, 0.3957,  ..., 0.5924, 0.4100, 0.5104])
Edge Score: torch.Size([9569])
Edge Score: tensor([1.0000, 0.5383, 1.0000,  ..., 0.4684, 0.4208, 0.5011])
Edge Score: torch.Size([6804])
Edge Score: tensor([1.0000, 0.6903, 0.5981,  ..., 0.4990, 0.3884, 0.5011])
Edge Score: torch.Size([6015])
Edge Score: tensor([1.0000, 0.6950, 0.6609,  ..., 0.3542, 0.4993, 0.5242])
Edge Score: torch.Size([5371])
Edge Score: tensor([1.0000, 0.5729, 0.8202,  ..., 0.3943, 0.5033, 0.5242])
Edge Score: torch.Size([4832])
Edge Score: tensor([1.0000, 0.6093, 0.6777,  ..., 0.4132, 0.4823, 0.5242])
Edge Score: torch.Size([4316])
Edge Score: tensor([1.0000, 0.8382, 0.7440,  ..., 0.5436, 0.4620, 0.5242])
Edge Score: torch.Size([3881])
Edge Score: tensor([1.0000, 0.8010, 0.7982,  ..., 0.5724, 0.4127, 0.5242])
Edge Score: torch.Size([3537])
Edge Score: tensor([1.0000, 0.7660, 0.7401,  ..., 0.4290, 0.4154, 0.5242])
Edge Score: torch.Size([3197])
Edge Score: tensor([1.0000, 0.8085, 0.9019,  .

In [24]:
G=nx.Graph()
G=nx.from_pandas_edgelist(el, 'sL', 'tL',create_using=nx.Graph())

In [25]:
color_map = {}
color_index = 0
for pair in i:
    for node in pair:
        color_map[node] = color_index
    color_index += 1
node_colors = [color_map[node] for node in G.nodes()]

In [26]:
import random
lol=df['id'].tolist()
a=random.choice(lol)
a=df[df['id']==a]

In [28]:
motifs_embeddings=x[len(x)-1][0]
a_emb=torch.tensor(a['emb'].values[0])   
a_emb.shape


In [ ]:
import torch
import torch.nn.functional as F

motifs_embeddings=x[len(x)-1][0]
a_emb=torch.tensor(a['emb'].values[0])   
a_emb.shape

def cosine_similarity(emb1, emb2):
    emb1_tensor = torch.tensor(emb1)
    emb2_tensor = torch.tensor(emb2)
    return F.cosine_similarity(emb1_tensor, emb2_tensor, dim=0).float()

ranks = {}
for i in range(motifs_embeddings.shape[0]):
    ranks[i] = cosine_similarity(motifs_embeddings[i], a_emb)

In [88]:
sorted_by_values_desc = dict(sorted(ranks.items(), key=lambda item: item[1], reverse=True))
print(sorted_by_values_desc)  

{28: tensor(0.8351), 103: tensor(0.8022), 140: tensor(0.7994), 111: tensor(0.7960), 148: tensor(0.7918), 109: tensor(0.7849), 68: tensor(0.7831), 142: tensor(0.7805), 116: tensor(0.7803), 147: tensor(0.7777), 81: tensor(0.7775), 1: tensor(0.7735), 145: tensor(0.7678), 141: tensor(0.7662), 144: tensor(0.7656), 508: tensor(0.7652), 204: tensor(0.7593), 406: tensor(0.7592), 38: tensor(0.7557), 121: tensor(0.7551), 16: tensor(0.7524), 571: tensor(0.7511), 40: tensor(0.7491), 117: tensor(0.7491), 26: tensor(0.7477), 47: tensor(0.7467), 157: tensor(0.7453), 2: tensor(0.7445), 172: tensor(0.7437), 405: tensor(0.7435), 65: tensor(0.7408), 48: tensor(0.7393), 128: tensor(0.7372), 50: tensor(0.7353), 815: tensor(0.7351), 151: tensor(0.7349), 462: tensor(0.7346), 64: tensor(0.7338), 201: tensor(0.7333), 86: tensor(0.7333), 152: tensor(0.7331), 3: tensor(0.7327), 554: tensor(0.7308), 216: tensor(0.7307), 781: tensor(0.7302), 170: tensor(0.7300), 149: tensor(0.7278), 601: tensor(0.7267), 156: tenso

In [89]:
import itertools
mot=cumilating(tensors)
count=0
stuff=[]
for j in sorted_by_values_desc:
    stuff+=mot[j]   
    count+=1
    if(count==5):
        break


28
[1251, 1412, 1417, 1482, 494, 1775, 86, 247]
False
103
[984, 2116, 255]
False
140
[773, 775, 776, 1929, 1422, 1423, 1934, 1171, 1939, 790, 1562, 671, 800, 1183, 1186, 681, 42, 682, 941, 305, 948, 569, 1853, 1985, 322, 1603, 2243, 454, 1865, 1098, 1740, 978, 1879, 1752, 1755, 2268, 990, 738, 2021, 236, 1005, 1900, 108, 497, 1394, 2164, 1658]
False
111
[1441, 2034, 423]
False
148
[1668, 1798, 1288, 1033, 18, 2070, 1432, 1840, 1457, 568, 1594, 62, 64, 720, 1110, 471, 2135, 1507, 1891, 2284, 2288, 882, 887, 1149]
False
109
[705, 450]
False
68
[611, 37, 1034, 275, 1950]
False
142
[2050, 4, 903, 1799, 1032, 1801, 1165, 528, 788, 1045, 536, 1050, 159, 1439, 1570, 421, 1789, 1838, 1790, 304, 2097, 690, 563, 949, 2102, 1081, 60, 317, 1220, 1093, 1480, 591, 594, 2010, 2140, 477, 479, 2016, 2027, 1388, 1402, 765, 126]
False
116
[1106, 1238]
False
147
[265, 1035, 271, 1809, 1300, 22, 284, 1692, 285, 1824, 425, 1963, 2219, 557, 45, 1210, 1727, 2240, 1728, 831, 2115, 2118, 1616, 1617, 97, 1896, 1

In [90]:
import itertools
mot=cumilating(tensors)
count=0
for j in sorted_by_values_desc:
    if(a["label"].values[0] in mot[j]):
        print(j)
        print(sorted_by_values_desc[j])
        print(mot[j])
        print(a["label"].values[0] in mot[j])
        count+=1


315
tensor(0.5538)
[241]
True


In [91]:
a['title'].values[0]

'Modular orientations of random and quasi-random regular graphs'

In [92]:
rec=df[df['label'].isin(stuff)]

In [93]:
rec

,id,title,year,n_citation,abstract,authors,references,emb,label
217,53e997ccb7602d9701fbd9c6,A local algorithm for finding dense subgraphs,2010,79,We present a local algorithm for finding dense...,"[{'id': '53f4376fdabfaee0d9b6e367', 'name': 'R...","[53e9a131b7602d9702a232e3, 53e997ccb7602d9701f...","[-0.41132298110000004, 0.15002977850000002, -0...",4
815,53e99833b7602d97020587f0,Quasi-random graphs.,1989,483,We introduce a large equivalence class of grap...,"[{'id': '5601bd4d45cedb3395eab3d4', 'name': 'F...","[53e9bd7bb7602d9704a1c870, 53e99833b7602d97020...","[-0.19190202650000002, 0.2339059711, -0.164615...",18
939,53e9984bb7602d970207a0d3,Worst-case equilibria,2009,470,In a system where noncooperative agents share ...,"[{'id': '54856a6bdabfaed7b5fa2146', 'name': 'E...","[53e997c6b7602d9701fb8204, 53e9a9a2b7602d97033...","[-0.2133031189, 0.3347230852, 0.00390023040000...",22
1415,53e998b0b7602d97020eca8f,The behavior of Wiener indices and polynomials...,2007,57,The sum of distances between all vertex pairs ...,"[{'id': '53fa0527dabfae9467e15b40', 'name': 'W...","[53e99cb5b7602d970256854e, 53e9a4dcb7602d9702d...","[-0.5040803552, 0.0318988562, -0.1550520808000...",37
1500,53e998c8b7602d9702103b0d,beta -Perfect Graphs.,1996,53,The class of β -perfect graphs is introduced. ...,"[{'id': '53f42eb8dabfaeb2acffb4c8', 'name': 'S...",[53e9ab89b7602d9703531c26],"[-0.4595018327, 0.2832813859, -0.1393359751, -...",42
...,...,...,...,...,...,...,...,...,...
71246,5c783b0c4895d9cbc6884f96,Algorithmic extensions of cheeger's inequality...,2011,27,We consider two generalizations of the problem...,"[{'id': '53f43487dabfaedce551d4dd', 'name': 'A...","[53e9980eb7602d9702024fb5, 53e99991b7602d97021...","[-0.3984740973, 0.0642142519, -0.0710348785000...",2240
71332,5c78df1c4895d9cbc6f736cd,On the family of r-regular graphs with Grundy ...,2013,6,"The Grundy number of a graph G, denoted by Γ(G...","[{'id': '53f43325dabfaedce550d7b2', 'name': 'N...","[53e99aa6b7602d9702320fd9, 53e99de8b7602d97026...","[-0.2726924419, -0.0486321077, -0.174098804600...",2243
73374,5ce2cebeced107d4c63021d7,On the geometric–arithmetic index of a graph,2019,7,"Very recently, Aouchiche and Hansen gave an up...","[{'id': '562cb25045cedb3398c9a066', 'name': 'Y...","[56ae9c290cf2a8c8f7148fca, 53e99aa6b7602d97023...","[-0.6259788275, -0.22742158170000001, 0.053956...",2268
73866,5d9edc4447c8f7664603988c,Approximation of biased Boolean functions of s...,2017,1,The influence of the kth coordinate on a Boole...,"[{'id': '53f4ce39dabfaeed20f8158a', 'name': 'N...","[53e9a102b7602d97029eea5e, 53e9a4f3b7602d9702e...","[-0.3337456882, 0.19411823150000002, -0.229343...",2284
